In [78]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 디버그 메시지 끄기

import tensorflow as tf
# gpu 사용 확인
print(tf.test.gpu_device_name())

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # tf.config.experimental.set_visible_devices(gpus[0], "GPU")
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

from angle_out import out
from tqdm.auto import tqdm
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import GRU, LSTM, Dense, Dropout, Embedding, Input
from keras import layers
import yaml
import xgboost as xgb
import numpy as np
import natsort
import matplotlib.pyplot as plt  # 데이터 시각화
import pickle
import json



def make_random_forest(
    n_estimators=100, max_depth=10, min_samples_split=2, random_state=0, verbose=1
):
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=random_state,
        verbose=verbose,
    )
    return model


def make_xgboost(verbose=2):
    model = xgb.XGBClassifier(verbosity=verbose)

    return model


def make_lstm():
    model = keras.Sequential()
    model.add(
        layers.LSTM(128, activation="relu", input_shape=(10, 14), return_sequences=True)
    )
    model.add(layers.Dropout(0.4))
    model.add(layers.LSTM(128, activation="relu", return_sequences=False))
    model.add(layers.Dense(32, activation="relu"))
    # model.add(layers.TimeDistributed(layers.Dense(13, activation="softmax")))
    model.add(layers.Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model


def make_gru():
    model = keras.Sequential()
    model.add(
        layers.GRU(
            128,
            activation="sigmoid",
            input_shape=(10, 14),
            dropout=0.2,
            return_sequences=False,
        )
    )
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

/device:GPU:0


In [ ]:
import numpy as np

data = np.load("label/BODYLOWER/20201024_dog-bodylower-000061.npy")
print(data[0].shape)
print(data[0])

In [86]:
# meta_data = []
from sklearn.model_selection import train_test_split

data = []
label = []
window = 10

path = "./label"
label_path = "./label"

with open("Zoo/config.yaml") as f:
    conf_yaml = yaml.load(f, Loader=yaml.FullLoader)

    bodyparts = []
    for part in conf_yaml["bodyparts"]:
        bodyparts.append(part)


label_parts = [
    ["nose", "neck_base", "neck_end"],
    ["neck_base", "neck_end", "back_base"],
    ["neck_end", "back_base", "back_middle"],
    ["back_base", "back_middle", "back_end"],
    ["back_middle", "back_end", "tail_base"],
    ["back_end", "tail_base", "tail_end"],
    ["back_base", "front_left_thai", "front_left_knee"],
    ["front_left_thai", "front_left_knee", "front_left_paw"],
    ["back_base", "front_right_thai", "front_right_knee"],
    ["front_right_thai", "front_right_knee", "front_right_paw"],
    ["back_end", "back_left_thai", "back_left_knee"],
    ["back_left_thai", "back_left_knee", "back_left_paw"],
    ["back_end", "back_right_thai", "back_right_knee"],
    ["back_right_thai", "back_right_knee", "back_right_paw"],
]

iii = 0

# print("data")
label_list = []
for list_ in natsort.natsorted(os.listdir(path)):
    print(f"{path}/{list_} > ")

    delta = f"{path}/{list_}"
    for delta_list in natsort.natsorted(os.listdir(delta)):
        alpha = f"{delta}/{delta_list}"
        if alpha.endswith(".csv"):
            continue
        elif alpha.endswith(".npy"):
            # print(f"{alpha} > ")
            jeta = np.load(alpha, allow_pickle=True)
            for index in range(jeta.shape[0]):
                # print(jeta[index].shape)
                if jeta[index].shape != (10, 14):
                    # print(jeta[index].shape)
                    # print(alpha)
                    continue
                data.append(jeta[index])
                if not list_ in label_list:
                    label_list.append(list_)
                # label.append([label_list.index(list_)])
                label.append([list_])

            pass
        iii += 1
# # print(np.shape(data))
print(f"data shape > {len(data)}")
x_tmp = np.array(data, dtype=np.float32) / 180.0
# # x_train = x_train.reshape(-1,4,10)
# print(f"shape > {y_tmp.shape}")
print(f"label shape > {np.shape(label)}")
print(f"label list > {label_list}")
y_tmp = np.array(label)
# y_tmp = tf.one_hot(y_tmp, len(label_list))
encoder = OneHotEncoder(sparse=False)
# encoder = OrdinalEncoder()
y_one_hot = encoder.fit_transform(y_tmp)
# y_one_hot = encoder.transform(y_tmp).toarray()

pickle.dump(encoder, open("model/encoder.pkl", "wb"))
with open("model/label.json", "w") as f:
    json.dump(label_list, f)
# y_train = tf.keras.utils.to_categorical(y_train, 13)


# x_tmp = x_tmp.reshape(-1, x_tmp.shape[1], x_tmp.shape[2])
# print(f"1 > {x_tmp[69520]}")

print(f"x shape > {x_tmp.shape}")
print(f"y tmp shape > {y_tmp.shape}")
print(f"y one hot shape > {y_one_hot.shape}")

./label/BODYLOWER > 
./label/BODYSCRATCH > 
./label/BODYSHAKE > 
./label/FEETUP > 
./label/FOOTUP > 
./label/HEADING > 
./label/LYING > 
./label/MOUNTING > 
./label/SIT > 
./label/TAILING > 
./label/TAILLOW > 
./label/TURN > 
./label/WALKRUN > 
data shape > 164557
label shape > (164557, 1)
label list > ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']
x shape > (164557, 10, 14)
y tmp shape > (164557, 1)
y one hot shape > (164557, 13)


/home/pieroot/miniconda3/envs/dlc/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [94]:
print(f"x_tmp : {x_tmp[0]}")
print(f"y_tmp : {y_tmp[0]}")
print(f"y_one_hot : {y_one_hot[0]}")

# y_recor = np.argmax(y_one_hot, axis=1).reshape(-1)
# print([y_one_hot[100000]])
y_record = encoder.inverse_transform([y_one_hot[0]])
print(f"y_recor : {y_record}")

x_tmp : [[ 1.74273056e+02 -6.61400070e+01 -1.91315735e+02  2.49659805e+01
   1.23338715e+02 -1.72189606e+02 -8.82498322e+01  5.57902832e+01
   2.10278244e+02 -2.39505875e+02  2.39476181e+02 -1.71361343e+02
   9.72429733e+01 -2.06787445e+02]
 [ 1.83927765e+02 -1.92780014e+02  1.41493317e+02  8.99997482e+01
  -2.51567581e+02  5.61558723e+01 -9.88318939e+01  8.97968197e+00
  -3.08014404e+02  2.33268890e+02  6.39648972e+01 -1.76758560e+02
  -1.62647228e+01  1.75269928e+02]
 [-9.55838470e+01  1.62709930e+02  1.63728638e+02 -7.49437027e+01
  -1.85835083e+02  5.54859161e+01  5.50701790e+01 -2.66508389e+01
   2.78239075e+02  2.56295090e+01  1.41971436e+02 -1.70454056e+02
  -8.80357437e+01  1.72771820e+02]
 [-3.20902023e+01  1.77211868e+02 -2.29433594e+02  2.21534088e+02
  -1.83918594e+02  6.55582199e+01  1.21866241e-01  1.11279039e+01
  -5.77022095e+01  2.77058319e+02  1.64837769e+02 -2.69937805e+02
   1.48875885e+02 -1.56362595e+02]
 [ 1.87684692e+02 -1.49975418e+02 -1.03884415e+02  2.0503724

In [89]:
# for i in y_tmp:
#     if True in np.isnan(i):
#         print('None')
#         break

# for i in x_tmp:
#     if True in np.isnan(i):
#         print('None')
#         break
x_train = x_tmp.reshape(-1, x_tmp.shape[1] * x_tmp.shape[2])
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
print(f"x shape > {x_train[:10]}")
# print(f"y shape > {y_tmp[1]}")

# X_train, X_test, Y_train, Y_test = train_test_split(
    # x_train[:10], y_tmp[:10], test_size=0.3, shuffle=True
# )

# print(f"x train > {X_train.shape} | y train > {Y_train.shape}")
# print(f"x test > {X_test.shape} | y test > {Y_test.shape}")
print(f"y list > {label_list}")

x shape > [[ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]
 [ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]
 [ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]
 ...
 [ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]
 [ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]
 [ 174.27306  -66.14001 -191.31573 ... -223.251    178.52277 -161.39864]]
y list > ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']


In [90]:
def make_lstm():
    model = keras.Sequential()
    model.add(
        layers.LSTM(128, activation="relu", input_shape=(10, 14), return_sequences=True)
    )
    model.add(layers.Dropout(0.4))
    model.add(layers.LSTM(128, activation="relu", return_sequences=False))
    model.add(layers.Dense(32, activation="relu"))
    # model.add(layers.TimeDistributed(layers.Dense(13, activation="softmax")))
    model.add(layers.Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model


def make_gru():
    model = keras.Sequential()
    model.add(
        layers.GRU(
            128,
            activation="sigmoid",
            input_shape=(10, 14),
            dropout=0.2,
            return_sequences=False,
        )
    )
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model


max_depth = 45
n_estimators = 100


def random_forest():
    x_train = x_tmp.reshape(-1, x_tmp.shape[1] * x_tmp.shape[2])
    model = make_random_forest(
        max_depth=max_depth, n_estimators=n_estimators, random_state=0, verbose=1
    )
    print(x_train.shape, y_one_hot.shape)
    model.fit(x_train, y_one_hot)
    model_params = model.get_params()
    print(model_params)
    pickle.dump(model, open(f"./model/rf_{max_depth}_{n_estimators}.pkl", "wb"))

    return model


def lstm():
    model = make_lstm()

    # model.compile(loss='binary_crossentropy',
    # optimizer='rmsprop',
    # metrics=['accuracy'])
    # model.compile(loss='mse', optimizer=Adam(0.01))

    print(model.summary())

    model.fit(x_tmp, y_one_hot, batch_size=64, epochs=100, verbose=1)

    return model


def gru():
    model = make_gru()

    model.fit(x_tmp, y_one_hot, batch_size=64, epochs=40, verbose=1)

    return model


def xgboost():
    x_train = x_tmp.reshape(-1, x_tmp.shape[1] * x_tmp.shape[2])
    model = make_xgboost()

    model.fit(x_train, y_one_hot)
    pickle.dump(model, open("./model/xgboost.pkl", "wb"))

    return model

# x_train=np.array(x_train) # (16,4,5,2) -> (16,4,10,1)

# x_train = x_train.reshape(-1, x_train.shape[1],x_train.shape[2], 1)

# print(x_train[0])
# print(y_tmp[0])

# model.fit(x_train, y_tmp, batch_size=1 , epochs=40, verbose=1)

# model.save('lstm_model.h5')
# model.save
# print("모델 저장 완려") #려~

# pred = model.predict(x_train) # 테스트 데이터 예측

# fig = plt.figure(facecolor='white')
# ax = fig.add_subplot(111)
# ax.plot(y_train, label='True')
# ax.plot(pred, label='Prediction')
# ax.legend()
# plt.show()

# print(x_tmp[1])
model = random_forest()
# model = lstm()
# model = gru()
# model = xgboost()

(164557, 140) (164557, 13)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   57.7s


{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 45, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 1, 'warm_start': False}


In [92]:
# with open(delta_label, "r") as label_json:
# label_tmp = json.load(label_json)[0]["pose"]

# model = keras.models.load_model('lstm_model.h5')
# lenght = len(x_train)
# faild = 0
# fail_list = []
# for i in tqdm(range(lenght), desc=f"오차 {faild/lenght}", mininterval=1):
#     x_tmp = [x_train[i]]
#     y_pred = model.predict(x_tmp)
#     if y_pred != y_tmp[i]:
#         faild += 1
#         print(f"실패 > {faild}/{i}")
#         fail_list.append(i)

# print(f"성공률 > {(1-faild/lenght)*100}")


def score_check():
    x_test = x_tmp.reshape(-1, x_tmp.shape[1] * x_tmp.shape[2])
    score = model.score(x_test, y_one_hot)
    # model.evaluate(x_tmp, y_tmp)
    print(f"score > {score}")
    print(f"model accuracy score > {accuracy_score(y_one_hot, model.predict(x_test))}")

    import time

    start = time.time()
    print(f"x_test[0] > {x_test[0]}")
    y_pred = model.predict([x_test[0]])
    # model.
    run_time = time.time() - start
    print(f"predict time > {run_time}")

    params = model.get_params()

    params["score"] = score
    params["pred_time"] = run_time
    # json_data = json.dumps(params)
    with open(f"./model/depth_{max_depth}_{n_estimators}.json", "w") as json_file:
        json.dump(params, json_file)


# print(f"실패한 갯수 > {faild/len(x_train)}")
# print(f"{x_train[0]}")


# y_pred = model.predict(x_tmp)
# print(f"예상치 > {y_pred}")
# print(f"실제값 > {y_tmp[0]}")

score_check()

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    4.2s


score > 1.0


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    4.0s


model accuracy score > 1.0
x_test[0] > [ 1.74273056e+02 -6.61400070e+01 -1.91315735e+02  2.49659805e+01
  1.23338715e+02 -1.72189606e+02 -8.82498322e+01  5.57902832e+01
  2.10278244e+02 -2.39505875e+02  2.39476181e+02 -1.71361343e+02
  9.72429733e+01 -2.06787445e+02  1.83927765e+02 -1.92780014e+02
  1.41493317e+02  8.99997482e+01 -2.51567581e+02  5.61558723e+01
 -9.88318939e+01  8.97968197e+00 -3.08014404e+02  2.33268890e+02
  6.39648972e+01 -1.76758560e+02 -1.62647228e+01  1.75269928e+02
 -9.55838470e+01  1.62709930e+02  1.63728638e+02 -7.49437027e+01
 -1.85835083e+02  5.54859161e+01  5.50701790e+01 -2.66508389e+01
  2.78239075e+02  2.56295090e+01  1.41971436e+02 -1.70454056e+02
 -8.80357437e+01  1.72771820e+02 -3.20902023e+01  1.77211868e+02
 -2.29433594e+02  2.21534088e+02 -1.83918594e+02  6.55582199e+01
  1.21866241e-01  1.11279039e+01 -5.77022095e+01  2.77058319e+02
  1.64837769e+02 -2.69937805e+02  1.48875885e+02 -1.56362595e+02
  1.87684692e+02 -1.49975418e+02 -1.03884415e+02  2

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [12]:
x_test = x_tmp.reshape(-1, x_tmp.shape[1] * x_tmp.shape[2])

print(np.shape(x_test[0]))

(140,)
